In [404]:
#########################################################
# IMPORTS NECESSARY FOR THE SCRIPT           
#########################################################
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import time
import statistics
import os
from dateutil.relativedelta import relativedelta

In [405]:
# ########################################################
# # UNCOMMENT SECTION BELOW TO USE THE TOTAL DATASET          
# ########################################################

# df = pd.read_csv("NY-data.csv") 
# paid = 0

In [406]:
#########################################################
# # UNCOMMENT SECTION BELOW TO USE THE PAID DATASET          
#########################################################

df = pd.read_csv("NY-data.csv") 
df = df[df["REVENUE"]>0]
paid = 1

In [407]:
#########################################################
# DONT NEED TO TOUCH THIS CODE BLOCK         
#########################################################
pd.options.mode.chained_assignment = None 
df.sort_values(["CUSTOMER_ID","CLASS_DATE"], ascending=[True,True], inplace = True)    ### SORT DATASET BY CUSTOMER_ID AND CLASS_DATE
folder = os.getcwd() +'/'   ##### CREATES FOLDER DIRECTORY TO STORE FILES

In [408]:
########################################################
# UNCOMMENT SECTION BELOW TO USE THE TRAINER FILTER         
########################################################

data_filter = 'TRAINER'
ny_filter = pd.read_csv("NY-trainers.csv")
ny_filter=np.array(ny_filter)

In [409]:
# #########################################################
# # # UNCOMMENT SECTION BELOW TO USE THE TIMESLOT FILTER          
# #########################################################

# data_filter = 'STUDIO'
# ny_filter = pd.read_csv("NY-studios.csv")
# ny_filter=np.array(ny_filter)

In [410]:
# #########################################################
# # # UNCOMMENT SECTION BELOW TO USE THE TIMESLOT FILTER          
# #########################################################

# data_filter = 'TIMESLOT'
# ny_filter = pd.read_csv("NY-timeslots.csv")
# ny_filter=np.array(ny_filter)

In [411]:
#########################################################
# STORES ALL THE CLASS DATES AND CUSTOMER_IDS IN 
# SEPARATE ARRAYS (DATE_ARRAY & ID_ARRAY)
# DONT NEED TO TOUCH THIS CODE BLOCK
#########################################################
id_array = df['CUSTOMER_ID'].tolist() 
date_array=[]   
for index,row in df.iterrows():    
    temp_str = str(row["CLASS_DATE"])
    date_array.append(datetime.strptime(temp_str, '%Y-%m-%d %H:%M').date())        #### CONVERTS DATE FROM STRING TO DATETIME

In [412]:
#########################################################
# CALULCATES THE TOTAL NUMBER OF CUSTOMERS DAILY (CUST_DAY)
# CALULATES TOTAL MONTHS THAT REGION HAS BEEN OPEN (TOTAL_MONTHS_OPEN)
# DONT NEED TO TOUCH THIS CODE BLOCK
#########################################################
sorted_dates=date_array.copy()          
sorted_dates.sort()                    #### SORTS THE DATES IN ORDER SO THAT WE CAN TOTAL DAILY COUNT (SORTED_DATES)
total_months_open = (sorted_dates[len(sorted_dates)-1].month)+(12*(sorted_dates[len(sorted_dates)-1].year-sorted_dates[0].year))    #### CALCULATES TOTAL MONTHS REGION HAS BEEN OPEN
df["CLASS DATE"] = sorted_dates
cust_per_day = df.pivot_table(index=(['CLASS DATE']), aggfunc='size')      ##### SEPARATES THE NUMBER OF CUSTOMERS BY DAY (CUST_PER_DAY)
cust_day = []
for i in range(len(cust_per_day)):
    cust_day.append(cust_per_day[i])             ###### FINAL ARRAY OF TOTAL CUSTOMERS DAILY

In [413]:
#########################################################
# CALUCLATES MONTHLY-FILTER STATS FOR CSV FILE(LATER)
# GENERATES LIST OF NUMBER OF FIRST TIMERS DAILY
#########################################################

total_avg_days_month = np.zeros((total_months_open,4))   #### TOTAL DAYS BETWEEN EACH CUSTOMER'S GROUP CLASSIFICATION BY MONTH
group_count_month = np.zeros((total_months_open,4))      #### TOTAL NUMBER OF CUSTOMERS IN EACH GROUP CLASSIFICATION BY MONTHLY
total_group4_monthly_classes = [0] * total_months_open   #### TOTAL NUMBER OF CLASSES FOR PEOPLE WITH 4+ CLASSES BY MONTHLY

filter_length= len(ny_filter)                                #### SIZE OF THE CURRENT FILTER FOR INDEXING (# OF TRAINERS OR # OF TIMESLOTS)
total_filter_length = (filter_length)*total_months_open        #### TOTAL SIZE OF THE FILTER BY MONTH, EX. 4 TRAINERS * 4 MONTHS = 16 DIF ROW VALUES (2D ARRAY)
total_avg_days_filter = np.zeros((total_filter_length,4))    #### TOTAL DAYS BETWEEN EACH CUSTOMER'S GROUP CLASSIFICATION BY MONTHLY-FILTER (2D ARRAY)
group_count_filter = np.zeros((total_filter_length,4))       #### TOTAL NUMBER OF CUSTOMERS IN EACH GROUP CLASSIFICATION BY MONTHLY-FILTER (2D ARRAY)
total_group4_filter_classes = [0] * total_filter_length      #### TOTAL NUMBER OF CLASSES FOR PEOPLE WITH 4+ CLASSES BY MONTHLY-FILTER (1D ARRAY)
total_filter_percent = [0] * total_filter_length             #### TOTAL NUMBER OF PEOPLE FOR THAT MONTHLY-FILTER *NOT* BY GROUP CLASSIFICATION, PERCENTAGE WILL BE CALCULATED LATER
filter_index = 0                                             #### RESETS INDEX OF THE MONTHLY-FILTER

date_cust_count = []   ### ARRAY COUNT OF NEW CUSTOMERS EVERYDAY (TO BE CALCULATED LATER)
list_of_dates = []    ### ARRAY OF ALL THE DATES FOR CUSTOMERS FIRST CLASS (NO DUPLICATES)
i=0     ### SET INDEX TO ZERO

#### LOOPS THROUGH ALL CUSTOMERS
while(i<len(id_array)-1):  
    
    list_of_dates.append(date_array[i])
    current_cust_id=id_array[i] #### SETS STARTING CUSTOMER_ID, CURRENT_CUST_ID IS COMPARED TO THE NEXT ID IN THE LIST (NEXT_CUST_ID)
    current_date = date_array[i] #### SETS STARTING CLASS_DATE ASSOCIATED WITH THAT CUSTOMER
    next_cust_id = id_array[i+1] #### SETS NEXT CUSTOMER_ID FOR COMPARISON
    next_date = date_array[i+1]  #### SETS NEXT CUSTOMER CLASS_DATE
    current_class_count = 1  #### STARTS CLASS COUNT AT 1 
    total_date_dif = 0.0    #### SETS TOTAL AVERAGE DAYS TO 0
    date_dif = 0.0          #### DIFFERENCE OF EACH AVERAGE
    current_year = 2017    #### STARTS CURRENT YEAR TO COMPARE
    current_month = 1     #### STARTS CURRENT MONTH TO COMPARE

    #####   DETERMINES WHICH FILTER WILL BE USED FOR GENERATING THE MONTHLY STATS, DO NOT NEED TO EDIT
    if(data_filter=='TRAINER'):
        filter = df[i:i+1].INSTRUCTOR.to_string(index=False)
    elif(data_filter=='TIMESLOT'):
        filter =  (datetime.strptime(str(df[i:i+1].CLASS_DATE.to_string(index=False)), ' %Y-%m-%d %H:%M').time()).strftime(" %H:%M:%S  ")
    else:
        filter = df[i:i+1].ROOM.to_string(index=False)
    ##########################
    
    ### IF CUSTOMER ONLY HAS 1 CLASS, THEY WILL NOT GO THROUGH THIS LOOP, current_cust_id IS THE ONLY OCCURRENCE OF THAT CUSTOMER_ID
    while (next_cust_id==current_cust_id):    ####   { LOOPS THROUGH ALL CLASSES FOR EACH CUSTOMER
        if (current_class_count==1):                ##### IF ITS THE CUSTOMERS FIRST CLASS, SAVE LOCATION, MONTH AND YEAR OF CLASS_DATE
                    
            for var in range(filter_length):                          #### FINDS THE INDEX OF THE FILTER OF THE CURRENT CUSTOMER
                filter_names = np.array(ny_filter[var:var+1])            #### SAVES THAT INDEX TO FILTER_INDEX, USED FOR INDEXING OF THE CUSTOMER CLASSIFICATION
                if (filter == filter_names.any()):
                    filter_index = var  

            while (current_year<current_date.year):    ##### STORES YEAR OF CUSTOMER'S FIRST CLASS
                current_year+=1
                current_month=1                       

            while(current_month<current_date.month):   #### STORES MONTH OF CUSTOMER'S FIRST CLASS
                current_month+=1                               
        ########## END OF IF STATEMENT #########3
        
                                  ###    CALCULATES TIME BETWEEN CLASSES AND ADDS TO TOTAL, ALSO INCREASES
        date_dif = (next_date-current_date)/timedelta(days=1)     #####  CLASS COUNT 
        total_date_dif =  total_date_dif + date_dif                    ###    }   
        current_date = next_date   ####   SETS FIRST DATE(TO COMPARE) TO THE NEXT DATE IN THE LIST
        if (i<len(id_array)-2):
            i+=1
        else:
            break
        next_cust_id = id_array[i+1]      ##### INCREASES CUSTOMER_ID TO NEXT IN LIST
        next_date = date_array[i+1]       ##### INCREASES 2ND DATE TO COMPARE TO NEXT DATE IN LIST
        current_class_count+=1     
########### LOOP THROUGH CUSTOMERS ENDS HERE, NOW ONTO CLASSIFYING EACH CUSTOMER BY CLASS COUNT

    if (current_class_count==1):     
                             ##### SETS CURRENT YEAR AND CURRENT MONTH FOR CUSTOMERS WITH 1 CLASS
        for var in range(filter_length):
                filter_names = np.array(ny_filter[var:var+1])
                if (filter == filter_names.any()):
                    filter_index = var
                    count3+=1
        while (current_year<current_date.year):
                current_year+=1
                current_month=1
            
        while(current_month<current_date.month):
                current_month+=1
                
        ########### MONTHLY STATS INDEXING BELOW
        
        group_count_month[(12*(current_year-2017))+(current_month-1)][0]+=1  #### INCREMENTS TOTAL GROUP COUNT FOR PEOPLE WITH 1 CLASS BY MONTH
        
        ############ MONTHLY-FILTER STATS INDEXING BELOW
        
        group_count_filter[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index][0]+=1    ##### INCREMENTS TOTAL GROUP COUNT FOR PEOPLE WITH 1 CLASS BY MONTH-FILTER
        total_filter_percent[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index]+=1     ### INCREMENTS TOTAL COUNT OF THAT MONTHLY-FILTER
    
    else:
        if (current_class_count>3):            ### IF CLASS COUNT IS 4+, CUSTOMER WILL BE CLASSIFIED HERE     
            temp_date_dif = total_date_dif/(current_class_count)   ### AVERAGE DAYS BETWEEN 4+ CLASSES FOR THIS PARTICULAR CUSTOMER
           
            ########### MONTHLY STATS INDEXING BELOW
            
            total_avg_days_month[(12*(current_year-2017))+(current_month-1)][3]+=temp_date_dif
            group_count_month[(12*(current_year-2017))+(current_month-1)][3]+=1             #### INCREMENT TOTAL GROUP4+ COUNT FOR THAT MONTH
            total_group4_monthly_classes[(12*(current_year-2017))+(current_month-1)] += current_class_count     #### ADD NUM OF CLASSES TO THE TOTAL 4+ CLASSES
           
            ############ MONTHLY-FILTER STATS INDEXING BELOW
            
            total_avg_days_filter[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index][3]+=temp_date_dif   ### ADD LATEST DATE_DIF TO TOTAL FOR THAT GROUP
            group_count_filter[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index][3]+=1             #### INCREMENT TOTAL GROUP4+ COUNT FOR THAT MONTH
            total_group4_filter_classes[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index] +=current_class_count  ### INCREMENT TOTAL GROUP4+ COUNT FOR THAT MONTH
            total_filter_percent[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index]+=1       ### INCREMENTS TOTAL COUNT OF THAT MONTHLY-FILTE
           
        else:                               #### IF CLASS COUNT IS 2 OR 3 (CURRENT_CLASS_COUNT), CUSTOMER WILL BE CLASSIFIED HERE
            temp_date_dif = total_date_dif/(current_class_count)
            
            ########### MONTHLY STATS INDEXING BELOW
            
            total_avg_days_month[(12*(current_year-2017))+(current_month-1)][current_class_count-1]+=temp_date_dif ##ADD LATEST DATE_DIF TO TOTAL
            group_count_month[(12*(current_year-2017))+(current_month-1)][current_class_count-1]+=1     #### INCREMENT TOTAL GROUP COUNT FOR THAT MONTH
            
            ############ MONTHLY-FILTER STATS INDEXING BELOW
            
            total_avg_days_filter[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index][current_class_count-1]+=temp_date_dif   ### ADD LATEST DATE_DIF TO TOTAL FOR THAT GROUP 
            group_count_filter[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index][current_class_count-1]+=1          ### INCREMENT TOTAL GROUP COUNT FOR THAT MONTH-FILTER
            total_filter_percent[((12*(current_year-2017))+(current_month-1))*filter_length+filter_index]+=1                                      ### INCREMENTS TOTAL COUNT OF THAT MONTHLY-FILTER
            
    ##### CLASS CLASSIFICATION LOOP ENDS, NOW GO TO NEXT CUSTOMER_ID
    
    
    if (i<len(id_array)-1):      ##### INDEX WILL INCREASE AS LONG AS WE'RE NOT AT THE END OF THE DATASET
        i+=1
    else:
        break

##### END OF THE LOOP THROUGH CUSTOMERS
list_of_dates.sort()   #### SORTS LIST OF CUSTOMER'S FIRST DATES
index=0               #### SETS INDEX AT ZERO
while(index<(len(list_of_dates)-2)):
    while (list_of_dates[index]==list_of_dates[index+1]):        ##### TAKES THE COUNT OF EACH UNIQUE CLASS DATE AND STORES IN DATE_CUST_COUNT
        if (index==len(list_of_dates)-1):
            break
        index+=1
    date_cust_count.append(list_of_dates.count(list_of_dates[index]))
    index+=1

In [414]:
#########################################################
# CALULATES THE MONTHLY COUNT FOR FIRST TIMERS
# MONTH_FIRST_TIMER_COUNT: ARRAY COUNT OF MONTHLY FIRST TIMERS
# GENERATES TEXT FILE THAT'S USED IN THE PREDICTIVE_MODEL FILE
#########################################################
index = 0
month_first_timers_count=[]

while(index<(len(list_of_dates)-1)):
    month_count = 0
    month_sum=0
    while (list_of_dates[index].month==list_of_dates[index+1].month):  
        if (index==len(list_of_dates)-2):
            break
        index+=1
        month_sum+=1
    month_count = month_count + month_sum 
    month_first_timers_count.append(month_count)
    index+=1
month_first_timers_count=np.array(month_first_timers_count)
if (paid==1):
    np.savetxt('monthly_first_timers_NY(PAID).txt', month_first_timers_count, delimiter =', ') 
else:
    np.savetxt('monthly_first_timers_NY(TOTAL).txt', month_first_timers_count, delimiter =', ') 

In [415]:
#########################################################
# TALLIES ACTUAL MONTHLY COUNT FOR ALL CUSTOMERS
# ACTUAL_MONTHLY_COUNT: ARRAY COUNT OF MONTHLY FIRST TIMERS
# GENERATES TEXT FILE THAT'S USED IN THE PREDICTIVE_MODEL FILE
#########################################################
actual_monthly_count=[]
index=0
while(index<len(sorted_dates)-2):
    month_sum=0
    while(sorted_dates[index].month==sorted_dates[index+1].month):
        month_sum+=1
        if (index>=len(sorted_dates)-2):
            break
        index+=1
    actual_monthly_count.append(month_sum)
    if (index>=len(sorted_dates)-2):
            break
    index+=1
if (paid==1):
    np.savetxt('actual_monthly_count_NY(PAID).txt', actual_monthly_count, delimiter =', ') 
else:
    np.savetxt('actual_monthly_count_NY(TOTAL).txt', actual_monthly_count, delimiter =', ') 

In [416]:
#########################################################
# GENERATES THE PERCENTAGE OF EACH PARTICULAR FILTER 
# THIS CODE BLOCK IS PART 1
# CURRENT_FILTER_SUM: THE SUM OF ALL THE FILTERS FOR THAT PARTICULAR MONTH
# TOTAL_FILTER_SUM: ARRAY OF ALL THE MONTHLY-FILTER SUMS, USED TO OBTAIN PERCENTAGES
# TOTAL_FILTER_PERCENT: THE TOTAL ARRAY OF ALL THE MONTHLY-FITLER PERCENTAGES
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################
current_filter_sum = 0 
total_filter_sum = []
count=1
index=1
while(index<total_filter_length):         ##### LOOPS THROUGH THE TOTAL LIST OF FILTERS
    if(count==filter_length):                     #### SUMS UP THE TOTAL COUNT FOR THAT FILTER, IF WE'RE AT THE END OF A MONTH, RESET SUM
        total_filter_sum.append(current_filter_sum) 
        current_filter_sum = 0
        count=0
    else:
        count+=1
        current_filter_sum = current_filter_sum + total_filter_percent[index-1]
        index+=1

In [417]:
for x in range(0,len(total_filter_sum)):
    for y in range(filter_length):
        total_filter_percent[filter_length*x+y] = total_filter_percent[filter_length*x+y]/total_filter_sum[x]         ##### PERCENTAGE OF EACH MONTHLY-FILTER FOR THAT MONTH

In [418]:
#########################################################
# THIS IS CODE BLOCK PART 2 FOR THE PERCENTAGES
# THE LAST/INCOMPLETE MONTH NEEDS TO BE DONE SEPARATELY
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################
temp_filter_sum=0
for i in range(len(total_filter_percent)-filter_length,len(total_filter_percent)):
    temp_filter_sum=temp_filter_sum+total_filter_percent[i]

for i in range(len(total_filter_percent)-filter_length,len(total_filter_percent)):
    total_filter_percent[i] = total_filter_percent[i]/temp_filter_sum 

In [419]:
#########################################################
# USES TOTALS FROM PREVIOUS CODE BLOCKS TO GENERATE FINAL MONTHLY-FILTER STATS
# FILTER_AVG_DAYS: FINAL 2D ARRAY OF AVERAGE DAYS BETWEEN CLASSES FOR EACH GROUP CLASSIFICATION BY MONTHLY-FILTER
# TOTAL_GROUP4_FILTER_CLASSES: ARRAY OF FINAL NUMBER OF 4+ CLASSES BY GROUP MONTHLY-FILTER
# GROUP_COUNT_FILTER: TOTAL *****PERCENTAGE***** OF THAT GROUP CLASSIFICATION BY MONTHLY-FILTER
# IGNORE THE WARNINGS AT THE END, THAT OCCURRS BECAUSE SOME COLUMNS ARE ZERO
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################

filter_avg_days = np.zeros((total_filter_length,4))     ### CREATE MATRIX FOR TOTAL MONTHS TO AVERAGE DAYS OF EACH GROUP AND FILTER CLASSIFICATION
for a in range(total_filter_length):
    for b in range(4):
        filter_avg_days[a][b] = total_avg_days_filter[a][b]/(group_count_filter[a][b])   ##### CALULATES AVERAGE DAYS FOR THAT CLASSIFICATION BY DIVIDING
                                                                                          #### TOTAL AVERAGE DAYS OF THAT GROUP BY TOTAL COUNT OF THAT GROUP
        
for x in range(total_filter_length):
    total_group4_filter_classes[x] = total_group4_filter_classes[x]/group_count_filter[x][3]  ##### CALCULATES AVERAGE # OF 4+ CLASSES FOR THAT
                                                                                                ### PARTICULAR MONTH-FILTER
total_sum_filter = []
for x in range(total_filter_length):
    total_temp = 0
    for y in range(4):                                   ##### CALCULATES THE SUM OF CUSTOMERS FROM EACH MONTH-FILTER
        total_temp += group_count_filter[x][y]           ##### EX. TOTAL_FILTER_SUM[0] IS THE TOTAL # OF CUSTOMERS OF JAN 2017 FROM THE FIRST TIMESLOT/TRAINER
    total_sum_filter.append(total_temp)                        
    
for x in range(total_filter_length):
    for y in range(4):                    
        group_count_filter[x][y] = group_count_filter[x][y]/total_sum_filter[x]      ##### DIVIDES TOTAL SUM BY TOTAL OF EACH GROUP TO GET % OF EACH GROUP
                                                                                      ##### IN THAT SPECIFIED MONTH-FILTER
            

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


In [420]:
#########################################################
# GENERATES 2 ARRAYS: 1. THE MONTHS/YEAR THAT NY HAS
# BEEN OPEN (MONTH_FILTER_LABELS) AND 2. THE NAME OF THE FILTER
# FOR THAT PARTICULAR MONTH (FILTER_LABELS)
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################
date = datetime.strptime('2017-01-09', '%Y-%m-%d')    ### START DATE
month_filter_labels = []
filter_labels = []
month_index=0
while (month_index<total_filter_length):
    for i in range(filter_length):
        month_filter_labels.append((str(date.strftime("%B") + ' ' + str(date.year))))      #### ADDS NEW MONTH ONCE WE'RE AT THE END OF THE FILTER LENGTH (NUMBER OF TRAINERS/TIMESLOTS)
        filter_labels.append(ny_filter[i])                                          #### ADDS THAT FILTER LABEL FOR VIEWING PURPOSES
        month_index+=1
    date = (date+relativedelta(months=1))    ##### INCREASES MONTH

In [421]:
#########################################################
# GENERATES THE MONTHS/YEAR THAT NY HAS
# BEEN OPEN (MONTH_LABELS)
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################
date = datetime.strptime('2017-01-09', '%Y-%m-%d')    ### START DATE
month_labels = []
month_index=0
for i in range(total_months_open):
    month_labels.append((str(date.strftime("%B") + ' ' + str(date.year))))      #### ADDS NEW MONTH ONCE WE'RE AT THE END OF THE FILTER LENGTH (NUMBER OF TRAINERS/TIMESLOTS)
    month_index+=1
    date = (date+relativedelta(months=1))    ##### INCREASES MONTH

In [422]:
#########################################################
# USED TO GENERATE 2D LIST FOR CSV FILE
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################
avg_days_2_classes_filter,avg_days_3_classes_filter,avg_days_4_classes_filter = [],[],[]
total_number_1_class_filter,total_number_2_classes_filter,total_number_3_classes_filter,total_number_4_classes_filter = [],[],[],[]
for index in range(total_filter_length):
    total_number_1_class_filter.append(group_count_filter[index][0])
    total_number_2_classes_filter.append(group_count_filter[index][1])                     ####  CREATES CSV FILE OF ALL THE MONTHLY STATISTICS ####
    total_number_3_classes_filter.append(group_count_filter[index][2])
    total_number_4_classes_filter.append(group_count_filter[index][3])
    avg_days_2_classes_filter.append(filter_avg_days[index][1])
    avg_days_3_classes_filter.append(filter_avg_days[index][2])
    avg_days_4_classes_filter.append(filter_avg_days[index][3])

In [423]:
#########################################################
# GENERATES CSV FILE FOR THE CHOSEN FILTER
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################

filter_stats = {'Month-Year': month_filter_labels,(data_filter + 'S'):filter_labels,(data_filter + ' Percent'):total_filter_percent,'CUSTOMER GROUP 1%':total_number_1_class_filter, 'CUSTOMER GROUP 2%':total_number_2_classes_filter, 
                                  'CUSTOMER GROUP 3%':total_number_3_classes_filter,
                                  'CUSTOMER GROUP 4+%':total_number_4_classes_filter, 'AVG TIME 2 CLASSES':avg_days_2_classes_filter,
                                  'AVG TIME 3 CLASSES':avg_days_3_classes_filter, 'AVG TIME 4+ CLASSES':avg_days_4_classes_filter, 
             'AVG NUMBER 4+ CLASSES':total_group4_filter_classes}
csv_filter_stats = (pd.DataFrame(data=filter_stats).fillna(0))
if (paid==1):
    csv_filter_stats.to_csv (folder+'NY-stats(' + data_filter +')(PAID).csv', index = None, header=True)
else:
    csv_filter_stats.to_csv (folder+'NY-stats(' + data_filter +')(TOTAL).csv', index = None, header=True)

In [424]:
csv_filter_stats

,Month-Year,TRAINERS,TRAINER Percent,CUSTOMER GROUP 1%,CUSTOMER GROUP 2%,CUSTOMER GROUP 3%,CUSTOMER GROUP 4+%,AVG TIME 2 CLASSES,AVG TIME 3 CLASSES,AVG TIME 4+ CLASSES,AVG NUMBER 4+ CLASSES
0,January 2017,[ AJ],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,January 2017,[ Aliyah],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,January 2017,[ Alvin],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,January 2017,[ Andy],0.059225,0.071942,0.237410,0.143885,0.546763,49.696970,63.833333,40.386458,28.671053
4,January 2017,[ Anthony],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,January 2017,[ Arnold],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,January 2017,[ Ash],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,January 2017,[ Ashley G],0.074137,0.103448,0.264368,0.086207,0.545977,29.717391,117.133333,41.455704,26.263158
8,January 2017,[ Beth],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,January 2017,[ Bradford R],0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [425]:
### CALCULATE MONTH PERCENTAGES AND AVERAGE DAYS

month_avg_days = np.zeros((total_months_open,4))     ### CREATE MATRIX FOR MONTHS(30) TO AVERAGE DAYS OF EACH GROUP CLASSIFICATION(4)
for a in range(total_months_open):
    for b in range(4):
        month_avg_days[a][b] = total_avg_days_month[a][b]/(group_count_month[a][b])   ##### CALULATES AVERAGE DAYS FOR THAT CLASSIFICATION BY DIVIDING
                                                                        #### TOTAL AVERAGE DAYS OF THAT GROUP BY TOTAL COUNT OF THAT GROUP
        
for y in range(total_months_open):
    total_group4_monthly_classes[y] = total_group4_monthly_classes[y]/group_count_month[y][3]  ##### CALCULATES AVERAGE # OF 4= CLASSES FOR THAT
                                                                                                ### PARTICULAR MONTH
total_sum = []
for total in range(total_months_open):
    total_temp = 0
    for tot in range(4):                                   ##### CALCULATES THE SUM OF CUSTOMERS FROM EACH MONTH
        total_temp += group_count_month[total][tot]       #### EX. TOTAL_SUM[0] IS THE TOTAL # OF CUSTOMERS OF JAN 2017
    total_sum.append(total_temp)                        
    
for r in range(total_months_open):
    for t in range(4):                    
        group_count_month[r][t] = group_count_month[r][t]/total_sum[r]      ##### DIVIDES TOTAL SUM BY TOTAL OF EACH GROUP TO GET % OF EACH GROUP
                                                                            #####    IN THAT SPECIFIED MONTH
##### GROUP_COUNT_MONTH IS A MATRIX OF THE PERCENTAGES OF EACH 4 GROUPS BY MONTH
##### MONTH_AVG_DAYS IS THE AVERAGE # OF DAYS BETWEEN CLASS FOR EACH GROUP CLASSIFICATION IN THAT SPECIFIED MONTH

In [426]:
#########################################################
# USED TO GENERATE 2D LIST FOR CSV FILE
# GENERATES MONTHLY-FILTER STATS
# DONT NEED TO EDIT THIS CODE BLOCK
#########################################################
avg_days_2_classes,avg_days_3_classes,avg_days_4_classes = [],[],[]
total_number_1_class,total_number_2_classes,total_number_3_classes,total_number_4_classes = [],[],[],[]
for index in range(total_months_open):
    total_number_1_class.append(group_count_month[index][0])
    total_number_2_classes.append(group_count_month[index][1])                     ####  CREATES CSV FILE OF ALL THE MONTHLY STATISTICS ####
    total_number_3_classes.append(group_count_month[index][2])
    total_number_4_classes.append(group_count_month[index][3])
    avg_days_2_classes.append(month_avg_days[index][1])
    avg_days_3_classes.append(month_avg_days[index][2])
    avg_days_4_classes.append(month_avg_days[index][3])
    
stats = {'MONTH-YEAR':month_labels,'CUSTOMER GROUP 1%':total_number_1_class, 'CUSTOMER GROUP 2%':total_number_2_classes, 
                                  'CUSTOMER GROUP 3%':total_number_3_classes,
                                  'CUSTOMER GROUP 4+%':total_number_4_classes, 'AVG TIME 2 CLASSES':avg_days_2_classes,
                                  'AVG TIME 3 CLASSES':avg_days_3_classes, 'AVG TIME 4+ CLASSES':avg_days_4_classes, 
             'AVG NUMBER 4+ CLASSES':total_group4_monthly_classes}
csv_stats = pd.DataFrame(data=stats)
if (paid==1):
    csv_stats.to_csv (folder+'NY-monthly-stats(PAID).csv', index = None, header=True)
else:
    csv_stats.to_csv (folder+'NY-monthly-stats(TOTAL).csv', index = None, header=True)

In [427]:
csv_stats

,MONTH-YEAR,CUSTOMER GROUP 1%,CUSTOMER GROUP 2%,CUSTOMER GROUP 3%,CUSTOMER GROUP 4+%,AVG TIME 2 CLASSES,AVG TIME 3 CLASSES,AVG TIME 4+ CLASSES,AVG NUMBER 4+ CLASSES
0,January 2017,0.109075,0.261184,0.094163,0.535577,24.154160,86.073906,39.748982,30.896579
1,February 2017,0.145101,0.290515,0.089196,0.475188,17.731351,84.731221,43.315341,24.775281
2,March 2017,0.177154,0.305698,0.091007,0.426141,18.806979,76.152324,43.957418,23.264259
3,April 2017,0.199268,0.318323,0.092879,0.389530,19.925287,76.144444,43.390976,22.260838
4,May 2017,0.196628,0.358627,0.094851,0.349895,14.656591,82.387302,44.216928,21.121343
5,June 2017,0.213564,0.334199,0.098701,0.353535,16.444301,76.754386,40.940685,21.355918
6,July 2017,0.237673,0.354886,0.087393,0.320047,17.837770,73.537162,39.539345,20.703875
7,August 2017,0.227394,0.361915,0.096882,0.313808,15.457846,63.774713,38.416644,18.833215
8,September 2017,0.241888,0.354966,0.089971,0.313176,16.585873,57.744080,32.651864,20.569859
9,October 2017,0.268422,0.348696,0.085845,0.297037,15.118010,52.415929,33.031677,19.294970
